# PyCity Schools Analysis

- analysis

---

In [2]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student data and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Merge data into single dataset
school_data_complete = pd.merge(student_data, school_data, on = "school_name", how="left")
school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

In [3]:
# Calculate total number of unique schools
school_count = school_data_complete["school_name"].nunique()
school_count

15

In [4]:
# Calculate toal number of students
student_count = school_data_complete["Student ID"].nunique()
student_count

39170

In [5]:
# Calculate total budget
# See ReadMe for groupby code note
unique_schools_df = school_data_complete.groupby(["school_name"]).head(1)

total_budget = unique_schools_df["budget"].sum()
total_budget

24649428

In [6]:
# Calculate average (mean) math score
average_math_score = school_data_complete["math_score"].mean()
average_math_score

78.98537145774827

In [7]:
# Calculate average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

81.87784018381414

In [8]:
# Calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [9]:
# Calculate the percentage of students who passed reading (reading scores greather than or equal to 70)
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [10]:
# Calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [11]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({ \
    "Total Schools":[school_count], "Total Students":[student_count], "Total Budget":[total_budget], \
        "Average Math Score":[average_math_score], "Average Reading Score":[average_reading_score], \
            "% Passing Math":[passing_math_percentage], "% Passing Reading":[passing_reading_percentage], \
                "% Overall Passing":[overall_passing_rate]})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


# School Summary

In [25]:
# Use the code provided to select the type per school from school_data
school_types = school_data.set_index(["school_name"])["type"]


In [13]:
# Calculate the total student count per school from school_data
per_school_counts = school_data.loc[:,['school_name', 'size']]
per_school_counts

,school_name,size
0,Huang High School,2917
1,Figueroa High School,2949
2,Shelton High School,1761
3,Hernandez High School,4635
4,Griffin High School,1468
5,Wilson High School,2283
6,Cabrera High School,1858
7,Bailey High School,4976
8,Holden High School,427
9,Pena High School,962


In [23]:
# Calculate the total school budget from school_data
per_school_budget = school_data.loc[:, ['school_name', 'budget']].copy()

# Calculate per capita spending
capita = per_school_budget["budget"] / per_school_counts["size"]

# Create a separate DataFrame for per_school_capita
per_school_capita = pd.DataFrame({
    'school_name': per_school_budget['school_name'],
    'per_capita': capita
})

# Formatting
per_school_budget["budget"] = per_school_budget["budget"].map("${:,.2f}".format)
per_school_capita["per_capita"] = per_school_capita["per_capita"].map("${:,.2f}".format)

# Display the DataFrames
per_school_budget, per_school_capita


(              school_name         budget
 0       Huang High School  $1,910,635.00
 1    Figueroa High School  $1,884,411.00
 2     Shelton High School  $1,056,600.00
 3   Hernandez High School  $3,022,020.00
 4     Griffin High School    $917,500.00
 5      Wilson High School  $1,319,574.00
 6     Cabrera High School  $1,081,356.00
 7      Bailey High School  $3,124,928.00
 8      Holden High School    $248,087.00
 9        Pena High School    $585,858.00
 10     Wright High School  $1,049,400.00
 11  Rodriguez High School  $2,547,363.00
 12    Johnson High School  $3,094,650.00
 13       Ford High School  $1,763,916.00
 14     Thomas High School  $1,043,130.00,
               school_name per_capita
 0       Huang High School    $655.00
 1    Figueroa High School    $639.00
 2     Shelton High School    $600.00
 3   Hernandez High School    $652.00
 4     Griffin High School    $625.00
 5      Wilson High School    $578.00
 6     Cabrera High School    $582.00
 7      Bailey High Sch

In [29]:
# Calculate the average test scores per school from school_data_complete
per_school_math = school_data_complete.groupby("school_name")["math_score"].mean()
per_school_reading = school_data_complete.groupby("school_name")["reading_score"].mean()

per_school_math, per_school_reading

(school_name
 Bailey High School       77.048432
 Cabrera High School      83.061895
 Figueroa High School     76.711767
 Ford High School         77.102592
 Griffin High School      83.351499
 Hernandez High School    77.289752
 Holden High School       83.803279
 Huang High School        76.629414
 Johnson High School      77.072464
 Pena High School         83.839917
 Rodriguez High School    76.842711
 Shelton High School      83.359455
 Thomas High School       83.418349
 Wilson High School       83.274201
 Wright High School       83.682222
 Name: math_score, dtype: float64,
 school_name
 Bailey High School       81.033963
 Cabrera High School      83.975780
 Figueroa High School     81.158020
 Ford High School         80.746258
 Griffin High School      83.816757
 Hernandez High School    80.934412
 Holden High School       83.814988
 Huang High School        81.182722
 Johnson High School      80.966394
 Pena High School         84.044699
 Rodriguez High School    80.744686
 Sh

In [38]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = school_data_complete[school_data_complete["math_score"] >= 70]["math_score"].count()
school_students_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby("school_name")["math_score"].count()

students_passing_math, school_students_passing_math


(29370,
 school_name
 Bailey High School       3318
 Cabrera High School      1749
 Figueroa High School     1946
 Ford High School         1871
 Griffin High School      1371
 Hernandez High School    3094
 Holden High School        395
 Huang High School        1916
 Johnson High School      3145
 Pena High School          910
 Rodriguez High School    2654
 Shelton High School      1653
 Thomas High School       1525
 Wilson High School       2143
 Wright High School       1680
 Name: math_score, dtype: int64)

In [39]:
# Calculate the number of students per school with reading scores of 70 or higher from school_data_complete
students_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70]["reading_score"].count()
school_students_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("school_name")["reading_score"].count()

students_passing_reading, school_students_passing_reading


(33610,
 school_name
 Bailey High School       4077
 Cabrera High School      1803
 Figueroa High School     2381
 Ford High School         2172
 Griffin High School      1426
 Hernandez High School    3748
 Holden High School        411
 Huang High School        2372
 Johnson High School      3867
 Pena High School          923
 Rodriguez High School    3208
 Shelton High School      1688
 Thomas High School       1591
 Wilson High School       2204
 Wright High School       1739
 Name: reading_score, dtype: int64)

In [43]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()


In [44]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100


In [50]:
# Create a DataFrame for the school summary
# Create a DataFrame for the school summary
per_school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": (students_passing_math / per_school_counts) * 100,
    "% Passing Reading": (students_passing_reading / per_school_counts) * 100,
    "% Overall Passing": ((students_passing_math + students_passing_reading) / (2 * per_school_counts)) * 100
})

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary


TypeError: unsupported operand type(s) for /: 'int' and 'str'